In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

In [2]:
source=requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup=BeautifulSoup(source,'lxml')
table_soup=soup.table.tbody.find_all('tr')
table_soup.remove(table_soup[0])
pc=list()
br=list()
ne=list()
for row in table_soup:
    try:
        pc.append(row.find_all('td')[0].text)
        br.append(row.find_all('td')[1].text)
        ne.append(row.find_all('td')[2].text.strip())
    except Exception as e:
        pass
toranto_df=pd.DataFrame({'Postalcode':pc,'Borough':br,'Neighborhood':ne})

In [3]:
toranto_df=toranto_df.drop(toranto_df[toranto_df['Borough']=='Not assigned'].index)

In [4]:
toranto_df.loc[toranto_df['Neighborhood']=='Not assigned','Neighborhood']=toranto_df.loc[toranto_df['Neighborhood']=='Not assigned','Borough']

In [5]:
toranto_gr=toranto_df.groupby("Postalcode")

In [6]:
toranto_gr

In [7]:
df=pd.DataFrame(columns=['Postalcode','Borough','Neighborhood'])
i=0
for grp in pd.unique(toranto_df["Postalcode"]):
    ni=list()
    grp_df=toranto_gr.get_group(grp)
    ni=grp_df['Neighborhood'].values
    #temp_df=pd.DataFrame({'Postalcode':[grp],'Borough':pd.unique(grp_df['Borough']),'Neighborhood':','.join(ni)})
    #print(temp_df)
    df.loc[i]=[grp,pd.unique(grp_df['Borough'])[0],','.join(ni)]
    i=i+1
df.head()

,Postalcode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [11]:
pip install geocoder # import geocoder


The following command must be run outside of the IPython shell:

    $ pip install geocoder # import geocoder

The Python package manager (pip) can only be used from outside of IPython.
Please reissue the `pip` command in a separate terminal or command prompt.

See the Python documentation for more informations on how to install packages:

    https://docs.python.org/3/installing/


In [9]:
!wget -o 'toranto_geodata.csv' https://cocl.us/Geospatial_data
data=pd.read_csv('https://cocl.us/Geospatial_data')
data.head()

'wget' is not recognized as an internal or external command,
operable program or batch file.


,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [12]:
lat_arr=list()
lon_arr=list()
for postal_code in df['Postalcode'].values:
    lat_arr.append(data.loc[data['Postal Code']==postal_code,'Latitude'].values)
    lon_arr.append(data.loc[data['Postal Code']==postal_code,'Longitude'].values)
df['Latitude']=lat_arr
df['Longitude']=lon_arr
df.head()

,Postalcode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,[43.7532586],[-79.3296565]
1,M4A,North York,Victoria Village,[43.725882299999995],[-79.31557159999998]
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",[43.6542599],[-79.3606359]
3,M6A,North York,"Lawrence Heights,Lawrence Manor",[43.718517999999996],[-79.46476329999999]
4,M7A,Queen's Park,Queen's Park,[43.6623015],[-79.3894938]


In [13]:
df.shape

(103, 5)